In [ ]:
import os
import joblib
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from emc_pre_engine import EmC_Pre_Generator
from emc_net_engine import EmC_Net_Generator
from emc_order_engine import EmC_Order_Box
from emc_loss import masked_mse
from logger_engine import EmC_Logger
from emc_aux_fun import netsummary

## Generate Data

In [ ]:
# Generate Random Targets

dim_imp,dim_ptrn = 24,37
sample_sizes = [4000,4000,4000,4000,4000]
minmaxes = [(0,0.0082),(0,0.0058),(0,0.0047),(0,0.0041),(0,0.0036)] # PEC Reflection 

seq_len = 20
dim_order = 2

emc_order_box = EmC_Order_Box("scaler/scale_0_0d01.save",norm_idx = True,env_dim = 2)
emc_order_box.generate_rnd(dim_ptrn,sample_sizes,minmaxes,seq_len,
                           inc_split = 13,
                           frq_split = 21,frq_range = (10,20))

t_size = 0.2
o_train,o_test,e_train,e_test,t_train,t_test,c_train,c_test = emc_order_box.split_orders(t_size,return_beam_number = True )
print(o_train,o_test,e_train,e_test,t_train,t_test,c_train,c_test)

In [ ]:
print(emc_order_box.env)

## Build and Load emc_net

In [ ]:
tf.keras.backend.clear_session()

batch_size = 100
emc_net_generator = EmC_Net_Generator(dim_order,dim_imp,dim_ptrn,env_dim = 2, target_seq_len = seq_len,batch_size = batch_size)
emc_net_generator.set_pre_conv(layer_num = 2,cluster = [3,2], channel = 1024, 
                               env_on_all_conv = False)
emc_net_generator.set_pre_fc(layer_num = 3,size = 1600)
emc_net_generator.set_inv_rnn(layer_num = 2, size = [512,1024],rnd_init = True)
emc_net_generator.set_inv_fc(layer_num = 3, size = 1024, actvfun = "relu",limit = True)

In [ ]:
folder_pre = "emc_pre_curve_incfqr"
uid_pre = "ek2ao3" # ID of the predictor just trained
para_pre = "epoch045-loss0.000401.hdf5" # Parameter to use

logger_pre = EmC_Logger(folder_pre,uid = uid_pre)
logger_pre.print_msg()

(_,dir_pre) = logger_pre.get_ids()
para_pre_name = f"{dir_pre}/network_para/{para_pre}"
print(f"Pre network to be loaded: {para_pre_name}")

In [ ]:
(emc_inv,emc_net) = emc_net_generator.create_EmC_Net()
emc_net_generator.load_pre_net(emc_net,para_pre_name)

emc_net.summary()
emc_inv.summary()

In [ ]:
foldername = "emc_net_curve_incfrq"

logger = EmC_Logger(foldername)
(uid,dirname) = logger.get_ids()

logger.write_msg(f"Network hyperparameter summary: \n{emc_net_generator}")
logger.write_msg(f"Network structure: \n{netsummary(emc_net)}")
data_msg = "Training on random dataset: \n"
data_msg += f"{dim_order} x {seq_len}\n"
data_msg += f"{sample_sizes}\n"
data_msg += f"{minmaxes}\n"
data_msg += f"test sizes: {t_size}"
logger.write_msg(data_msg)

In [ ]:
n_epochs = 120
learning_rate = 5e-5

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=dirname+"/network_para/epoch{epoch:03d}-loss{val_loss:.6f}.hdf5",
    monitor="val_loss",verbose=1, save_best_only=True,save_weights_only=True, mode="min")
emc_net.compile(loss=masked_mse, optimizer=opt)

logger.write_msg(f"# of epoch = {n_epochs} \n Batch size = {batch_size} \n Optimazation with Adam, learning rate {learning_rate}")

In [ ]:
logger.write_msg(f"Training starts")
history = emc_net.fit([o_train,e_train], t_train, validation_data=([o_test,e_test], t_test), 
                      epochs=n_epochs, batch_size=batch_size, shuffle=True,
                      callbacks=[checkpoint])
logger.write_msg(f"Training finished")
np.savetxt(f"{dirname}/loss.txt",history.history['loss'])
np.savetxt(f"{dirname}/val_loss.txt",history.history['val_loss'])
logger.write_msg(f"Losses saved")

In [ ]:
num_epoch=list(range(1,n_epochs+1))
plt.figure(figsize=(6,4))

#Plot loss vs epoch
plt.rc('font', size=14) 
plt.plot(num_epoch, history.history['loss'])
plt.plot(num_epoch, history.history['val_loss'])
plt.legend(('Training Set','Validation Set'), fontsize=12, loc=1)
plt.xlabel('Epoch #', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.xlim(1,n_epochs)
plt.ylim(0,0.05)
plt.xticks(list(range(0,n_epochs,100)))

plt.show()

In [ ]:
freq_scaler = joblib.load("scaler/scale_4d3_4d7.save")
inc_scaler = joblib.load("scaler/scale_m30_30.save")
ptrn_scaler = joblib.load("scaler/scale_0_0d01.save")
vol_scaler = joblib.load("scaler/scale_0_18.save")

t_predict = emc_net.predict([o_test,e_test],batch_size = batch_size)

theta = np.arange(0, np.pi+np.pi/36, np.pi/36)

plt.figure(figsize=(15,15))

for n_example in range (0,25):
    y_example = ptrn_scaler.inverse_transform(t_test[n_example,:].reshape(1, -1))
    y_predictn = ptrn_scaler.inverse_transform(t_predict[n_example,:].reshape(1, -1))
    a_example = inc_scaler.inverse_transform(e_test[n_example,:].reshape(1,-1))[0,0]
    f_example = freq_scaler.inverse_transform(e_test[n_example,:].reshape(1,-1))[0,1]
    
    #plt.subplot(5,5,n_example+1,projection = 'polar')
    plt.subplot(5,5,n_example+1)
    plt.rc('font', size=8)
    plt.plot(theta, y_predictn.reshape(dim_ptrn,1))
    plt.plot(theta, y_example.reshape(dim_ptrn,1),'o')
    plt.ylim(0,0.008)
    plt.rc('font', size=8)
    plt.text(1,0.006,f"{a_example:.0f} deg", fontsize=12)
    plt.text(1,0.005,f"{f_example:.2f} GHz", fontsize=12)
    if n_example == 0: plt.legend(('Prediction','Target'), fontsize=12, loc=3)